In [38]:
import requests
import re
from bs4 import BeautifulSoup
import math
import pandas as pd
import time
import random

https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

# Target URL
Follow the steps to produce a target URL:
1. Goto Linkedin and search for a job with any filters you want.
2. Open up the Developer inspector (Use Google Chrome) and scroll down until the next set of jobs load.
3. In the network tab there should be a "search" request when it loads the new jobs.
4. Grab that URL in the search request.
5. replace the last NUMBER in the URL with a {} in order to utilize a .format() for looping later. ie 'start=0' -> 'start={}'

In [10]:
target_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_JT=F&f_E=2&f_WT=2&currentJobId=3664087381&position=1&pageNum=0&start={}'

# Job ID Loop
Used to get the Job ID to appened to the job URL for crawling

In [65]:
def job_id_scrape(target_url, num_of_jobs):
    l=[] #Empty list to store job IDs. These IDs are used to create the URLs for specific job searches later
    a = 0 #Used for the URL page number
    pattern = 'jobPosting:(?P<job_id>\d+)' #Regex patter to get the job ID
    for i in range(0, math.ceil(num_of_jobs/25)): #Use math.ceil() to round off the loop, the numerator can be found by looking at the filters when building the target URL.
        res = requests.get(target_url.format(a)) #Get request against the target URL and a .format() in order to provide specific page number.
        a += 25
        soup = BeautifulSoup(res.text, 'html.parser')
        all_jobs = soup.find_all('li') #The 'li' tag contains the job ID
        #print(len(all_jobs)) #Debug
        for j in range(0, len(all_jobs)): #Loop through all b4 'li' list
            job_id = re.search(pattern, str(all_jobs[j])) #Using Regex to search for the ID
            l.append(int(job_id.groups()[0])) #Appened the ID and casting it as int for later use.
    return l

In [118]:
job_ids = job_id_scrape(target_url, 25)

In [119]:
job_ids

[3695796378,
 3664087381,
 3695796357,
 3624206900,
 3632360327,
 3695790928,
 3693849036,
 3603332905,
 3627030054,
 3695809550,
 3685291387,
 3695792590,
 3695829145,
 3653339410,
 3648295510,
 3674709878,
 3692136204,
 3693865409,
 3695792577,
 3686134894,
 3683935408,
 3635642191,
 3673021599,
 3684851788]

# Job Loop

In [114]:
def job_scrape(job_id_list):
    df = {'job_title' : [], 'company' : [], 'job_info' : [], 'when_posted' : [], 'num_applicants' : [],
          'seniority' : [], 'employment_type' : [], 'job_function' : [], 'industry' : []}
    target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}/' #format of the URL. The {} is where the jobID gets formated to.
    for i in range(0, len(job_id_list)): # Main loop
        res_job = requests.get(target_job_url.format(job_id_list[i]))
        soup = BeautifulSoup(res_job.text, 'html.parser')
        try:
            df['job_title'].append(soup.find('div',{'class':'top-card-layout__entity-info'}).find('a').text.strip()) #Title
        except:
            df['job_title'].append(None)
        try:
            df['company'].append(soup.find('div',{'class':'top-card-layout__card'}).find('a').find('img').get('alt')) #Company
        except:
            df['company'].append(None)
        try:
            df['job_info'].append(soup.find('div',{'class':'show-more-less-html__markup'}).text.strip()) #Bulk, might need to find a way to clean this up.
        except:
            df['job_info'].append(None)
        try:
            df['when_posted'].append(soup.find('span',{'class':'posted-time-ago__text'}).text.replace('\n', '').strip()) #When was it posted
        except:
            df['when_posted'].append(None)
        num_of_apps1 = soup.find('figcaption',{'class':'num-applicants__caption'})  #number of applicants scape attempt1
        if num_of_apps1 == None: #Check first soup, if it fails check second soup
            num_of_apps2 = soup.find('span', class_='num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet') #number of applicants scrape attempt2
            if num_of_apps2 == None: #Check second soup, if it fails, append None
                df['num_applicants'].append(None) #if nosoup works just append None
            else:
                df['num_applicants'].append(num_of_apps2.text.replace('\n', '').strip()) #If first soup failed and second soup passed, append second soup
        else:
            df['num_applicants'].append(num_of_apps1.text.replace('\n', '').strip()) #If first soup works, append first soup
        job_attributes = soup.find('ul',{'class':'description__job-criteria-list'}) #Job attributes
        if job_attributes == None: #Check to see if there is soup. If not, pass.
            pass
        else:
            job_attributes = job_attributes.text.replace('\n', '').strip()
            pattern = 'Seniority level\s+(?P<seniority>.*?)\s\s+Employment type\s+(?P<job_type>.*?)\s\s+Job function\s+(?P<job_function>.*?)\s\s+Industries\s+(?P<industry>.*)'
            job_attributes = re.search(pattern, job_attributes).groups()
            df['seniority'].append(job_attributes[0]) #Seniority
            df['employment_type'].append(job_attributes[1]) #Employment type
            df['job_function'].append(job_attributes[2]) #Job function
            df['industry'].append(job_attributes[3]) #Industry
        time.sleep(random.choice(range(3,8,1))) # Sleep to avoid IP block
    return pd.DataFrame(df)

In [120]:
df1 = job_scrape(job_ids)

In [121]:
df1.head()

,job_title,company,job_info,when_posted,num_applicants,seniority,employment_type,job_function,industry
0,Data Scientist (Entry Level),Pattern Learning AI - Career & Tech Recruitmen...,This is a remote position.Data Scientist (Entr...,2 hours ago,66 applicants,Entry level,Full-time,Engineering and Information Technology,Software Development
1,Data Scientist,Analytica,Analytica is seeking a remoteData Scientistto ...,1 month ago,Over 200 applicants,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
2,Data Scientist (Entry Level) - US,Pattern Learning AI - Career & Tech Recruitmen...,This is a remote position.Data Scientist (Entr...,2 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and Information Technology,Software Development
3,Data Scientist,QuantumBricks,Job Title: Data scientistLoc: RemoteExp: 9+ Yr...,2 months ago,Over 200 applicants,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting
4,Data Scientist - Strategy,"Agero, Inc.","About Agero:Wherever drivers go, we’re leading...",3 weeks ago,Over 200 applicants,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting


In [122]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        24 non-null     object
 1   company          24 non-null     object
 2   job_info         24 non-null     object
 3   when_posted      24 non-null     object
 4   num_applicants   24 non-null     object
 5   seniority        24 non-null     object
 6   employment_type  24 non-null     object
 7   job_function     24 non-null     object
 8   industry         24 non-null     object
dtypes: object(9)
memory usage: 1.8+ KB
